# Import Multiple Datasets in csv format

1.    데이터 전처리 (Data_Description 파일 기준)

제외조건1)    20번 Propeller RPM 기준 20% 이상 급변하는 가속/감속 구간(약 20분) 데이터 제외

제외조건2)    2번 Validity가 FALSE인 데이터 제외

제외조건3)    11번 Sea Depth는 분석에서 제외

제외조건4)    5번 Speed over Ground가 10knots 이하 또는 16.5knots 이상인 데이터 제외

제외조건5)    14번 True Wind Speed가 Beaufort Scale 4 이상인 데이터 제외

* https://en.wikipedia.org/wiki/Beaufort_scale

제외조건6)    27번 ~ 40번 (기상데이터) 중 데이터 값이 ‘9999’ 또는 ‘-9999’ 인 데이터 or 결측치(NaN) 제외

# 제외조건 1) 급속구간 부터 20분간 데이터 삭제

In [ ]:
%reset

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
filepaths = ['9421984_201709.csv', '9421984_201710.csv','9421984_201711.csv','9421984_201801.csv','9421984_201802.csv','9421984_201803.csv','9421984_201804.csv','9421984_201805.csv','9421984_201806.csv','9421984_201807.csv','9421984_201808.csv','9421984_201809.csv','9421984_201810.csv','9421984_201811.csv']
ShipData = pd.concat(map(pd.read_csv, filepaths))

In [ ]:
# 이렇게 해야 index 0,1,2,3,4,5,6,7,8,9 가 겹치지 않는다. (이 전에 그냥 concat하면 index=0인 obs가 여러개 - 문제됨 -행렬 방식이 아님 )
ShipData.index = np.arange(len(ShipData))
ShipData.index

In [ ]:
# 앞뒤 전후 차이 계산한 새로운 variable 정의 
ShipData["RPM change"]=abs(ShipData["Propeller RPM"].diff()*100/ShipData["Propeller RPM"])

In [ ]:
# 20% 초과 포인트: 3576개
sum(ShipData["RPM change"]>20)

In [ ]:
start=list(ShipData[(ShipData["RPM change"]>20)].index)
print(start)

In [ ]:
last=list(ShipData[(ShipData["RPM change"]>20)].index+120)

In [ ]:
def moonsu (i) :  
    a = np.arange(start[i],last[i],1)
    return a
# np.arange(start[1],last[1],1)  # 시작, 끝(포함하지 않음), 단계

In [ ]:
list_to_append = []
for i in range(3576) : 
    result = moonsu(i)
    list_to_append.append(result)

In [ ]:
abnormal=np.asarray(list_to_append)

In [ ]:
abnormal2=abnormal.flatten()

In [ ]:
#중복제거한 인덱스 만들기 
import numpy as np
y = np.bincount(abnormal2)
ii = np.nonzero(y)[0]
len(ii)

In [ ]:
#ShipData2: 비정상 운행 구간 삭제 (90900개 관측치)
ShipData2=ShipData.drop(ShipData.index[ii])
ShipData2.shape

# 제외조건 2) Validity가 FALSE인 데이터 제외

In [ ]:
ShipData3=ShipData2[ShipData2.Validity != False]

In [ ]:
ShipData3.Validity.describe()

# 제외조건 3) Sea Depth 분석에서 제외

In [ ]:
ShipData4=ShipData3.drop(['Voyage','From','To','Status','Validity','Sea Depth'],axis=1)

In [ ]:
ShipData4.head()

In [ ]:
ShipData4.columns #Sea Depth 제외 확인 

# 제외조건 4)  5번 Speed over Ground가 10knots 이하 또는 16.5knots 이상인 데이터 제외

In [ ]:
ShipData5=ShipData4.drop(ShipData4[(ShipData4["Speed over Ground"]<=10)|(ShipData4["Speed over Ground"]>=16.5)].index)

In [ ]:
ShipData5["Speed over Ground"].describe() # 검토완료 

# 제외조건 5) 14번 True Wind Speed가 Beaufort Scale 4 이상인 데이터 제외( 풍속 5.5m/s 이상 ) 

In [ ]:
ShipData6=ShipData5.drop(ShipData5[(ShipData5["True Wind Speed"]>=5.5)].index)

In [ ]:
ShipData6["True Wind Speed"].describe() # 검토완료

# 제외조건6)    27번 ~ 40번 (기상데이터) 중 데이터 값이 ‘9999’ 또는 ‘-9999’ 인 데이터 or 결측치(NaN) 제외

In [ ]:
# 1.  결측치 제외 : any는 행의 성분에 NaN이 하나라도 있으면 그 행을 제거 하라는 뜻 - 1189개 남음
ShipData7=ShipData6.dropna(how="any")

In [ ]:
ShipData7.describe()

In [ ]:
ShipData7[ShipData7.Latitude !=999]

In [ ]:
ShipData7[ShipData7.Latitude !=-999]

In [ ]:
ShipData7[ShipData7.Latitude !=-9999]

In [ ]:
ShipData7[ShipData7.Latitude !=9999]

In [ ]:
# 검토: 모든변수 (39개 변수) 에서 999 or -999/ 9999 or -9999가 있는지 확인 -없음 
for i in range(39):
    print(len(ShipData7[ShipData7.iloc[:,i] !=9999])) 

In [ ]:
for i in range(39):
    print(len(ShipData7[ShipData7.iloc[:,i] !=-9999])) #-999,9999,-9999

In [ ]:
for i in range(39):
    print(len(ShipData7[ShipData7.iloc[:,i] !=999])) 

In [ ]:
for i in range(39):
    print(len(ShipData7[ShipData7.iloc[:,i] !=-999])) 

# 최종 전처리된 Dataframe csv파일로 내보내기 

In [ ]:
ShipData7.to_csv('ship.csv') #최종 전처리된 Dataframe csv파일로 내보내기 